## gold.rst_base_empresarial

### 1. Objetivo
Processar e integrar dados de empresas, estabelecimentos e Simples Nacional para gerar uma **tabela GOLD agregada por CNAE e UF**.  

A tabela resultante (`gold.rst_base_empresarial`) fornece informações sobre:
- Empresas ativas
- Distribuição por CNAE
- Capital social agregado
- Participação em MEI e Simples Nacional

---

### 2. Fontes de Dados

#### SILVER
| DataFrame | Fonte | Descrição |
|-----------|-------|-----------|
| `df_estab` | `silver.antigo_estabelecimentos` | Informações de estabelecimentos: CNPJ, UF, CNAE, matriz/filial, situação cadastral |
| `df_emp` | `silver.antigo_empresas` | Capital social e porte da empresa |
| `df_simples` | `silver.antigo_simples` | Opção MEI e Simples Nacional |

### GOLD (Dimensões)
| DataFrame | Fonte | Descrição |
|-----------|-------|-----------|
| `df_dim_uf` | `gold.dim_uf` | Informações da UF: sigla, nome, código |
| `df_dim_cnae` | `gold.dim_cnaes` | Informações do CNAE: código e descrição |

---

### 3. Fluxo do Notebook

1. **Configuração de performance**
   - `partitionOverwriteMode = dynamic`
   - `autoBroadcastJoinThreshold = 10MB`

2. **Leitura das dimensões GOLD**
   - UF: `cod_uf`, `sg_unidade_federativa` (maiuscula e trim)
   - CNAE: `cod_cnae`, `descricao`

3. **Leitura e filtragem SILVER**
   - Seleção de colunas relevantes
   - Filtro de empresas ativas (`st_cadastral = '02'`)

4. **Junções (Joins)**
   - `df_base` + `df_emp` via CNPJ
   - Resultado + `df_simples` via CNPJ
   - Integração com dimensões UF e CNAE (broadcast join)

5. **Preparação de colunas**
   - Seleção de colunas finais: UF, CNAE, indicador matriz/filial, capital social, MEI e Simples
   - Remoção de registros com UF ou CNAE nulos

6. **Agregação para GOLD**
   - Chave: `sg_unidade_federativa`, `cod_cnae`
   - Métricas:
     | Coluna | Descrição |
     |--------|-----------|
     | `empresas_ativas` | Contagem de empresas ativas |
     | `empresas_matriz` | Contagem de empresas matriz |
     | `capital_social_total` | Soma do capital social (nulos = 0) |
     | `qtd_mei` | Quantidade de empresas MEI |
     | `qtd_simples` | Quantidade de empresas Simples Nacional |

7. **Carga na tabela GOLD**
   - Formato Delta, modo overwrite, mergeSchema = true
   - Otimização física para BI:
     ```sql
     OPTIMIZE gold.rst_base_empresarial ZORDER BY (cod_cnae)```

---

### 4. Estrutura da Tabela GOLD

| Coluna | Tipo | Descrição |
|--------|------|-----------|
| `sg_unidade_federativa` | string | Sigla da UF (ex: SP, RJ) |
| `cod_cnae` | string/int | Código CNAE |
| `empresas_ativas` | long | Total de empresas ativas |
| `empresas_matriz` | long | Total de empresas matriz |
| `capital_social_total` | decimal(20,2) | Soma do capital social |
| `qtd_mei` | long | Quantidade de empresas MEI |
| `qtd_simples` | long | Quantidade de empresas Simples Nacional |

---

### 5. Resultados Esperados

- Tabela **consolidada e agregada** de empresas por CNAE e UF
- Permite análises como:
  - Distribuição de empresas ativas por estado e setor
  - Matriz vs filial
  - Capital social agregado
  - Quantidade de MEI e Simples Nacional

---

### 6. Observações

- Broadcast join usado para otimizar joins pequenos
- Nulos de capital social tratados como 0

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [0]:
# configs de performance

spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 10 * 1024 * 1024 )

In [0]:
# paths

path_silver = "abfss://silver@storagedatanexus.dfs.core.windows.net/"
target_table = "gold.rst_base_empresarial"
dim_cnaes_table = "gold.dim_cnaes"
dim_uf_table = "gold.dim_uf"

In [0]:
# Leitura Dimensões (GOLD)
df_dim_uf = spark.read.table("gold.dim_uf") \
    .select(
        F.col("cod_uf"),
        F.upper(F.trim("sigla_uf")).alias("sg_unidade_federativa")
    )

df_dim_cnae = spark.read.table("gold.dim_cnaes") \
    .select(
        F.col("cod_cnae"),
        F.col("descricao")
    )

# Leitura - SILVER
df_estab = spark.read.format("delta").load(path_silver + "antigo_estabelecimentos") \
    .select(
        "cnpj", 
        F.col("uf").alias("sg_unidade_federativa"), 
        F.col("cod_cnaes").alias("cod_cnae"), 
        "identificador_mtz_fil", 
        "st_cadastral"
    )

df_emp = spark.read.format("delta").load(path_silver + "antigo_empresas") \
    .select(
        "cnpj", 
        F.col("capital_social").alias("capital_social_total"), 
        "porte_empresa"
    )

df_simples = spark.read.format("delta").load(path_silver + "antigo_simples") \
    .select(
        "cnpj",
        "opcao_mei",
        "opcao_simples"
    )

In [0]:
# Filtro empresas ativas=02
df_base = df_estab.filter(F.col("st_cadastral") == "02")

# Joins
df_join = df_base \
    .join(F.broadcast(df_emp), "cnpj", "left") \
    .join(F.broadcast(df_simples), "cnpj", "left")

# Preparação das colunas (grão)
df_prep = df_join.select(
    "sg_unidade_federativa",
    "cod_cnae",
    "identificador_mtz_fil",
    "capital_social_total",
    "opcao_mei",
    "opcao_simples"
).filter(
    F.col("sg_unidade_federativa").isNotNull() &
    F.col("cod_cnae").isNotNull()
)

df_union_dim = df_prep \
    .join(F.broadcast(df_dim_uf), "sg_unidade_federativa", "left") \
    .join(F.broadcast(df_dim_cnae), "cod_cnae", "left")

# Agregação de grão com dimensões
df_gold = df_union_dim.groupBy(
    "sg_unidade_federativa",
    "cod_cnae"
).agg(
    F.count("*").alias("empresas_ativas"),
    F.sum(
        F.when(F.col("identificador_mtz_fil") == "1", 1).otherwise(0)
    ).alias("empresas_matriz"),
    F.sum(
        F.coalesce("capital_social_total", F.lit(0))
    ).cast("decimal(20,2)").alias("capital_social_total"),
    F.sum(
        F.when(F.col("opcao_mei") == "S", 1).otherwise(0)
    ).alias("qtd_mei"),
    F.sum(
        F.when(F.col("opcao_simples") == "S", 1).otherwise(0)
    ).alias("qtd_simples")
)

# Carga na tabela GOLD
df_gold.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwrite", "true").option("mergeSchema", "true") \
    .saveAsTable(target_table)

# Otimização física para o PBI
spark.sql(f"OPTIMIZE {target_table} ZORDER BY (cod_cnae)")

In [0]:
spark.sql("""
    SELECT * 
    FROM gold.rst_base_empresarial
    WHERE COD_CNAE = '4923001'
    --WHERE SG_UNIDADE_FEDERATIVA IN ("16402320")
""").display(truncate=False)